In [1]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import scipy.optimize as optimize
from scipy.optimize import brenth

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
pi=4*np.arctan(1.)

In [2]:
def U(x):
    return x**2+0.02*(np.cos(167*x)+np.sin(73*x))

def dU(x):
    return 2*x+0.02*(-167*np.sin(167*x)+73*np.cos(73*x))

def T(x):
    return 1.-(dU(x)/g1)**2

def ddU(x):
    return 2.+0.02*(-167**2*np.cos(167*x)-73**2*np.sin(73*x))
def Ueff(x):
    return U(x)+T(x)/g2

def dUeff(x):
    return dU(x)*(1.-2*ddU(x)/(g2*g1**2))

In [3]:
def LL(x):
    return dU(x)+g1

def UL(x):
    return dU(x)-g1

def integrand(x):
    return dUeff(x)/T(x)

In [4]:
def root(F, x1, x2, div, it):
    X=np.linspace(x1,x2, div)
    roots=np.array([])
    for i in range(div-1):
        if F(X[i])*F(X[i+1])<=0:
            p1=X[i]
            p2=X[i+1]
            r=brenth(F, p1, p2, maxiter=it)
            roots=np.append(roots, r)
    return roots

In [5]:
def integral2(F, X):
    l=len(X)
    Func=np.zeros(l)
    id0=int(0.5*l)
    xmin=X[id0]
    Func[id0]=0
    for i in range(1,l):
        if i<=id0:
            x0 = X[id0-i+1]
            x1 = X[id0-i]
            Func[id0-i]=integrate.quad(F, x0, x1)[0]
            Func[id0-i]=Func[id0-i]+Func[id0-i+1]
        else:
            x0 = X[i-1]
            x1 = X[i]
            Func[i]=integrate.quad(F, x0, x1)[0]
            Func[i]=Func[i]+Func[i-1]
    return Func

In [6]:
g1=5.5
g2=100

L1=-0.5*g1-0.02*167
L2=-0.5*g1+0.02*167

U1=0.5*g1-0.02*167
U2=0.5*g1+0.02*167

In [7]:
rmax=root(UL, U1,U2, 10000, 300)
rmin=root(LL, L1,L2, 10000, 300)

In [8]:
if min(rmax)>max(rmin):
    xmax=min(rmax)
    xmin=max(rmin)
else: 
    imax=np.argmin(np.absolute(rmax))
    imin=np.argmin(np.absolute(rmin))
    xmin=np.minimum(rmax[imax], rmin[imin])
    xmax=np.maximum(rmax[imax], rmin[imin])

In [9]:
r=root(dUeff, xmin, xmax, 30000, 300)
x=np.linspace(xmin,xmax, 2000)
x=np.append(x,r)
x=np.sort(x)

In [10]:
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(x, U(x), color='red')
plt.xlim([xmin,xmax])
plt.title(r'Potencial de Zwanzig', fontsize=18)
plt.xlabel('$x$', fontsize=18)
plt.ylabel('$U(x)$', fontsize=18)


plt.subplot(1,2,2)
plt.plot(x, T(x), color='red')
plt.title(r'Temperatura efectiva', fontsize=18)
plt.plot([min(x), max(x)],[0,0], 'k--')
plt.xlabel('$x$', fontsize=18)
plt.ylabel('$T(x)$', fontsize=18)

plt.show()

In [11]:
plt.figure(figsize=(14, 6))

plt.subplot(1,2,1)
plt.plot(x, Ueff(x), color='red')
plt.xlim([xmin,xmax])
plt.title('Potencial efectivo', fontsize=18)
plt.xlabel('$x$', fontsize=18)
plt.ylabel(r'$U_{eff}(x)$', fontsize=18)


plt.subplot(1,2,2)
plt.plot(x, dUeff(x), color='red')
plt.plot([min(x), max(x)], [0,0], 'k--')
plt.xlim([xmin,xmax])
plt.title('Derivada del potencial efectivo', fontsize=18)
plt.xlabel('$x$', fontsize=18)
plt.ylabel(r'$dU_{eff}(x)/dx$', fontsize=18)

plt.show()

In [22]:
#product=np.array([0.01, 0.1, 0.5, 1, 2., 10])
#g1=3.5
g2=100
product=np.array([0.1, 0.5, 2, 10])
G2=np.array([0.1, 5.5, 100])
G1=np.array([0.1, 5.5, 8])
#G2=np.array([0.5])#product/g1**2
color=['purple', 'blue', 'green', 'yellow', 'orange', 'red']

In [23]:
plt.figure(figsize=(30, 6))
for i in range(len(G1)):
    g1=G1[i]

    L1=-0.5*g1-0.02*167
    L2=-0.5*g1+0.02*167

    U1=0.5*g1-0.02*167
    U2=0.5*g1+0.02*167

    rmax=root(UL, U1,U2, 10000, 300)
    rmin=root(LL, L1,L2, 10000, 300)

    if min(rmax)>max(rmin):
        xmax=min(rmax)
        xmin=max(rmin)
    else: 
        imax=np.argmin(np.absolute(rmax))
        imin=np.argmin(np.absolute(rmin))
        xmin=np.minimum(rmax[imax], rmin[imin])
        xmax=np.maximum(rmax[imax], rmin[imin])
        
    r=root(dUeff, xmin, xmax, 30000, 300)
    x=np.linspace(0.9*xmin, 0.9*xmax, 5000)
    x=np.append(x, r)
    x=np.sort(x)
    plt.subplot(1, 3, i+1)
    for j in range(len(G2)):
        g2=G2[j]    
        label=g2*g1**2
        inte=integral2(integrand, x)
        inte=-g2*inte
        prob=np.exp(inte)
        prob[prob==np.nan]=0
        Z=np.trapz(prob, x)
        #x=x-min(x)
        #x=x/max(x)
        plt.plot(x/g1, prob/Z,label=r'$g_2 g_1^2=%.2f$'%label, color=color[j])    
    #plt.plot(x/g1, prob/Z,label='$g_2g_1^2=%.2f$'%label, color=color[i])
    #plt.plot(x/g1, prob*Ueff(x)/Z, color='k')

    plt.xlabel(r'$x/g_1$', fontsize=18)
    plt.ylabel('$P(x)$', fontsize=18)
    plt.title(r'$g_1=%.2f$'%g1, fontsize=18)
    plt.legend(loc='best', frameon=0)
plt.show()

In [115]:
G1=np.linspace(50,54, 100)
g2=0.1
v1=np.zeros(len(G1))
v2=np.zeros(len(G1))
nroot=np.zeros(len(G1))

In [116]:
for i in range(len(G1)):
    g1=G1[i]

    L1=-0.5*g1-0.02*167
    L2=-0.5*g1+0.02*167

    U1=0.5*g1-0.02*167
    U2=0.5*g1+0.02*167

    rmax=root(UL, U1,U2, 10000, 300)
    rmin=root(LL, L1,L2, 10000, 300)

    if min(rmax)>max(rmin):
        xmax=min(rmax)
        xmin=max(rmin)
    else: 
        imax=np.argmin(np.absolute(rmax))
        imin=np.argmin(np.absolute(rmin))
        xmin=np.minimum(rmax[imax], rmin[imin])
        xmax=np.maximum(rmax[imax], rmin[imin])
        
    r=root(dUeff, xmin, xmax, 30000, 300)
    v1[i]=xmin
    v2[i]=xmax
    nroot[i]=len(r)
    
    

In [117]:
Interval=np.absolute(v2-v1)
P1=2*pi/167.
P2=2*pi/73.
Nroot=Interval*(1./P1+1./P2)

In [120]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 2)
plt.plot(Interval, nroot, label=r'N\'umero de ra\'ices')
#plt.plot(Interval, Nroot, label=r'$I(1/P_1+1/P_2)$')
plt.ylim([0, 1.1*np.maximum(max(nroot), max(Nroot))])
plt.xlabel('Longitud del dominio', fontsize=18)
plt.legend(loc='best', frameon=0, fontsize=18)

plt.subplot(1,2, 1)
plt.plot(G1, v1, color='blue', label=r'$x_{min}$')
plt.plot(G1, v2, color='red', label=r'$x_{max}$')
plt.xlabel('$g_1$', fontsize=18)
plt.ylabel('$x$', fontsize=18)
plt.legend(loc='best', frameon=0, fontsize=18)
plt.title(r'Dominio como funci\'on de $g_1$', fontsize=18)

plt.show()